Data Partition as Training and Test sets

In [3]:
import os, sys
import image_norm_test as myData;
import numpy as np
from sklearn.cross_validation import train_test_split
seed = 3071986   # set random seed
rng = np.random.RandomState(seed)
import re
import fetch_file

# fetch all the file in the directory path  defined#
pwd = os.pardir;
accessPath = r"%s\patch_database" %pwd;
filelist = []
print(accessPath)


# in this way we can extract only certain number image of each samples, ex. 2 means only 2 images of one texture
filelist , lable = fetch_file.fetchFile(accessPath , 4)
print(filelist[0])
print(filelist[100])
filelist = np.array(filelist).transpose()[0:100];
lable = np.array(lable).transpose()[0:100];
print(lable)


..\patch_database
..\patch_database\sample01_002_3x3
..\patch_database\sample26_002_3x3
[ 1  1  1  1  2  2  2  2  3  3  3  3  4  4  4  4  5  5  5  5  6  6  6  6  7
  7  7  7  8  8  8  8  9  9  9  9 10 10 10 10 11 11 11 11 12 12 12 12 13 13
 13 13 14 14 14 14 15 15 15 15 16 16 16 16 17 17 17 17 18 18 18 18 19 19 19
 19 20 20 20 20 21 21 21 21 22 22 22 22 23 23 23 23 24 24 24 24 25 25 25 25]


In [2]:
from sklearn import preprocessing


rng = np.random.RandomState(seed)
permutation = rng.permutation(len(filelist))

#print(permutation)
#print(filelist[permutation])
X, y = filelist[permutation], lable[permutation]


# split our data into half of train data and half of text data randomly.
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.75, random_state=seed)


print(train_y)
print(test_y)
#print(test)


[ 9  2  7  5 24 19  2 10  2 21 20 25 23 23 24 16  3 25  1 17 20 16 12 23  9
 19  6 15 12 14 25 10 15  8 17 22 17 11 25 10 21  5  1  4  6 18 20  4 13 14
 18  9  5  9 20 21 24 18  8  7 12 22  7 16  8 18  1 13  3 16 12 22 23  3  8]
[19 13 11  2 10 21 11 24  3  1 14  7 15  4 13  6 17 11 22 15 14  5  6  4 19]


Start to get the features learned from feed in data

Retrive centroids ( features ) from file stored in disk

In [7]:
import get_feature

centroid = []
count = 0
temp = []
n = 3*3
num_data = 10000
clusters = [16,32,64,128]
#cluster = 16
#centroid_file = r"%s\%dfeature_%d.txt" %(pwd ,  cluster , num_data);
for cluster in clusters:
    centroid_file = r"%s\25_4_3_%d.txt" %(pwd , cluster)
    centroid.append(get_feature.read_features(centroid_file))
'''
data = np.loadtxt(centroid_file,delimiter="\n")
for i in range(0 , len(data)):
    temp.append(data[i])
    count += 1
    if(count == n):
        centroid.append(temp)
        temp = []
        count = 0
'''
print(centroid[0])
print(len(centroid[0]))

[[-0.359201245835, -0.350276323095, -0.284874560613, -0.387853317701, -0.356222505484, -0.265256232066, -0.382704706903, -0.336148081943, -0.244915257572], [0.358806866422, 0.290237064273, 0.279483142322, 0.341993511392, 0.295504326746, 0.315070116924, 0.35871147642, 0.356818418472, 0.386476904451], [-0.148784868061, 0.164467159988, 0.442070713355, -0.131500140521, 0.248641382045, 0.541956821763, -0.0695002573605, 0.294172120952, 0.539617973866], [-0.37811067742, -0.182680751809, 0.262972398401, -0.36602944445, 0.000716159993457, 0.476164862946, -0.207526841013, 0.192900624991, 0.55992543393], [0.211746974514, 0.364218791337, 0.373559949287, 0.261235304683, 0.405884763544, 0.384645076997, 0.275612693403, 0.367548912751, 0.301609240793], [0.49121356796, 0.349911768499, 0.0140163857025, 0.511247189891, 0.315396451391, -0.040035183695, 0.459491759478, 0.238122601938, -0.0760792258937], [-0.270118685166, 0.188059162761, 0.550179916507, -0.344981907231, -0.22451350832, 0.0962181922777, -0.2

Get the bag of features for each patch 

In [ ]:
from my_vlad import my_vlad

# will load data as the patch size defined , 3 means 3*3 = 9 for each patch, and will return the dictionary included:
# 'data'  (one patch)  , 'target' (the sample of this patch belongs to ) , 'filename' (the file comes from)
bofs = []
vlad = my_vlad(centroid[0])
for file in train_X:
    mydata,y = myData.load_sig_data(file , 3)
    bofs.append(vlad.get_vlad(mydata['data']).flatten() )
#print(bofs[0])
#print(X_train_tf)
#print(train_y)


Used the bag of features of each image and the known lable to do classification with knn 

In [20]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import grid_search


#knn_init = KNeighborsClassifier()
#parameters = {'n_neighbors':[ 5, 10 , 15]}
#knn = grid_search.GridSearchCV(knn_init, parameters)

knn = KNeighborsClassifier(n_neighbors = 2)

knn.fit(bofs, train_y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=2, p=2,
           weights='uniform')

In [ ]:
bofs_test = []
for file in test_X:
    mydata,y = myData.load_sig_data(file , 3)
    bofs_test.append(vlad.get_vlad(mydata['data']).flatten() )
#print(bofs)


In [24]:
import numpy as np
predicted = knn.predict(bofs_test)
print(predicted)
print(test_y)
knn.score(bofs_test,test_y)

 
#np.mean(predicted == test_y)  

[14 13  8  3  4  3  8  3  8  3 14  7 13  4 13  3  6  8  1 13 14  1  6  8 13
  5  6 20  5  3 13  6  8  6  3  8  1  8  4  3  3  3 22 22  3 20  6  5  8  9]
[19 13 11  2 10 21 11 24  3  1 14  7 15  4 13  6 17 11 22 15 14  5  6  4 19
  9  2  7  5 24 19  2 10  2 21 20 25 23 23 24 16  3 25  1 17 20 16 12 23  9]


0.22

When used k = 16 and with 4 images of each sample, and with 25 samples, so totally have 100 images (knn neighbor with 2)

In [5]:
import run
run.run(centroid , 0 , train_X , train_y ,test_X , test_y, method='knn' , n_nb = 2)

C:\Users\howfungirl\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.19.
  DeprecationWarning)


0.35999999999999999

When used k = 32 and with 4 images of each sample, and with 25 samples, so totally have 100 images  (knn neighbor with 2)

In [6]:
import run
run.run(centroid , 1 , train_X , train_y ,test_X , test_y, method='knn' , n_nb = 2)

C:\Users\howfungirl\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.19.
  DeprecationWarning)


0.20000000000000001

When used k = 16 and with 4 images of each sample, and with 25 samples, so totally have 100 images  (knn neighbor with 8)

In [8]:
import run
run.run(centroid , 0 , train_X , train_y ,test_X , test_y, method='knn' , n_nb = 8)

0.040000000000000001

When used k = 16 and with 4 images of each sample, and with 25 samples, so totally have 100 images  (knn neighbor with 1)

In [9]:
import run
run.run(centroid , 0 , train_X , train_y ,test_X , test_y, method='knn' , n_nb = 1)

0.47999999999999998

Used LinearSVM 

In [4]:
from Choose_target import Choose_Target

from sklearn import preprocessing


rng = np.random.RandomState(seed)
permutation = rng.permutation(len(filelist))

#print(permutation)
#print(filelist[permutation])
X, y = filelist[permutation], lable[permutation]
select1 = Choose_Target(X, y)
select1.select(1)
X1 = select1.X
y1 = select1.y

# split our data into half of train data and half of text data randomly.
train_X, test_X, train_y, test_y = train_test_split(X1, y1, train_size=0.75, random_state=seed)




In [ ]:
import run
run.run(centroid , 0 , train_X , train_y ,test_X , test_y, method='LinearSVM' )